In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.15.1'

In [6]:
!pip install tensorflow-decision-forests==1.8.0

In [7]:
#Funcion de normalizacion para los datos (Pasar valor de los pixeles de 0-255 a 0-1)
#Hace que la red aprenda mejor y mas rapido
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #Aqui lo pasa de 0-255 a 0-1
  return imagenes, etiquetas

In [8]:
!pip install tensorflow-datasets

In [9]:
import tensorflow_datasets as tfds
import math



In [10]:
#Descargar set de datos de MNIST (Numeros escritos a mano, etiquetados)
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

#Obtenemos en variables separadas los datos de entrenamiento (60k) y pruebas (10k)
datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

2025-10-05 12:17:16.346129: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-10-05 12:17:16.346312: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-10-05 12:17:16.346330: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-10-05 12:17:16.346399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-05 12:17:16.346467: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
datos

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [12]:
#Etiquetas de las 10 categorias posibles (simplemente son los numeros del 0 al 9)
nombres_clases = metadatos.features['label'].names

In [13]:
nombres_clases

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [14]:
#Normalizar los datos de entrenamiento y pruebas con la funcion que hicimos
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

In [15]:
datos_entrenamiento

<_MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [16]:
!pip install keras-tuner

In [17]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
from kerastuner import RandomSearch
from tensorflow import keras


In [18]:
num_ej_entrenamiento = metadatos.splits['train'].num_examples
num_ej_pruebas = metadatos.splits['test'].num_examples
TAMANO_LOTE = 32

In [19]:
# Shuffle y batching
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)


In [20]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

### KerasTuner

In [21]:
def build_model(hp):
    modelo = keras.Sequential()
    modelo.add(keras.layers.Flatten(input_shape=(28,28,1)))
    
    # Capas densas entre 1 y 3
    for i in range(hp.Int('num_layers', 1, 3)):
        modelo.add(keras.layers.Dense(
            units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=64),
            activation='relu',
            kernel_regularizer=keras.regularizers.l2(hp.Float(f'l2_{i}', 0.0, 0.01, step=0.001))
        ))
        modelo.add(keras.layers.Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))
    
    modelo.add(keras.layers.Dense(10, activation='softmax'))
    
    modelo.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return modelo

In [22]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,            # puedes aumentar según tu tiempo
    executions_per_trial=1,
    directory='mnist_tuner',
    project_name='red_regular'
)

In [23]:
tuner.search(
    datos_entrenamiento,
    epochs=10,
    validation_data=datos_pruebas,
    steps_per_epoch=math.ceil(num_ej_entrenamiento/TAMANO_LOTE),
    callbacks=[early_stop]
)

Trial 2 Complete [00h 04m 39s]
val_accuracy: 0.9061999917030334

Best val_accuracy So Far: 0.9061999917030334
Total elapsed time: 00h 09m 30s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
192               |320               |units_0
0.007             |0.001             |l2_0
0.4               |0.4               |dropout_0
0.0029185         |0.0013673         |learning_rate



Epoch 1/10


: 

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
print("Mejores hiperparámetros encontrados:")
print(best_hp.values)

In [ ]:
historial = best_model.fit(
    datos_entrenamiento,
    epochs=20,
    validation_data=datos_pruebas,
    steps_per_epoch=math.ceil(num_ej_entrenamiento/TAMANO_LOTE),
    callbacks=[early_stop]
)
